# Growth rate estimates

Code to read in information per country on total number of cases and number of days since case 100
Then to fit the data, obtaining the date of inflection and the initial growth rate

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import csv
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

import pandas 

%matplotlib inline

In [153]:
#Fit functions
def logistic_model(x,a,b,c):
    return c/(1+np.exp(-(x-b)/a))

def exponential_model(x,a,b,c):
    return a*np.exp(b*(x-c))

def linear_fit(x,m,c):
    return x*m + c

In [154]:
#Calculate chi-squared goodness of fit
def chisquare(data,expct):
    return sum((data-expct)**2 / expct)

In [189]:
#Function to obtain growth rate from data file
#Input: 
# inputfile = days since 100 cases , total number of cases, csv format
# country = name of country of interest
# daterange = boolean : option for a user specified range of days
# daystart = 0  : first day for fit period
# dayend = 100 : last day for fit period
def ObtainGrowthRate(inputfile,country,daterange=False,daystart=0,dayend=100):
    
    #Read in data file 
    df = pandas.read_csv(inputfile)
    
    #Define date range to fit. Default = all days
    mask = np.ones(len(df["Days since 100"]),dtype=bool)
 
    #Mask to only consider days with data
    dayend = df["Days since 100"][np.argmax(df[country])]
    mask = (df["Days since 100"] < dayend) & (df[country][mask] > 0.)

    #Adjust if date range is specified
    if daterange:
        mask = (df["Days since 100"] > daystart) & (df["Days since 100"] < dayend)
    
    #Check for sufficient data points:
    if len(df[country][mask]) < 5:
        print("Insufficient data for",country)
        return 0., 0., 0.
    
    #Perform a linear fit to the data
    popt, pcov = curve_fit(linear_fit, df["Days since 100"][mask],np.log10(df[country][mask]))
    chi2 = chisquare(df[country][mask],10.**(df["Days since 100"][mask]*popt[0] + popt[1]))

    inflection_date = 0.
    logchi2 = 0
    #Need minimum number of approx. ten days for a logistic fit
    if len(df["Days since 100"][mask]) > 15.:
        #Perform a logistic fit to the data
        try:
            log_fit, log_cov = curve_fit(logistic_model,df["Days since 100"][mask],df[country][mask],
                                 p0=[2.5,1,max(df[country][mask])],maxfev=1200)
            logchi2 = chisquare(df[country][mask],logistic_model(df["Days since 100"][mask],*log_fit))
            
            inflection_date = log_fit[1]
        except RuntimeError:
            print("Logistic function fit failed for",country)
            

    if (chi2 < logchi2) or (inflection_date > max(df["Days since 100"][mask])) or not inflection_date:
        growthrate = popt[0]
        print("linear fit preferred for",country,"growth rate",growthrate)
        return growthrate, 0., inflection_date

    print("logistic fit preferred for",country)
    print("inflection date",inflection_date)
    
    #Perform linear fits pre and post inflection date to compare growth rates
    mask1 = mask & (df["Days since 100"] < inflection_date)
    mask2 = mask & (df["Days since 100"] > inflection_date)

    growthrate = 0.
    growthrate2 = 0.
    
    #Check for sufficient data points:
    if sum(mask1) > 5:
        popt, pcov = curve_fit(linear_fit, df["Days since 100"][mask1],np.log10(df[country][mask1]))
        chi2 = chisquare(df[country][mask1],10.**(df["Days since 100"][mask1]*popt[0] + popt[1]))
        growthrate = popt[0]
    
    if sum(mask2) > 5:
        popt2, pcov2 = curve_fit(linear_fit, df["Days since 100"][mask2],np.log10(df[country][mask2]))
        chi2_2 = chisquare(df[country][mask2],10.**(df["Days since 100"][mask2]*popt2[0] + popt2[1]))
        growthrate2 = popt2[0]
        
    return growthrate, growthrate2, inflection_date
    

In [190]:
ObtainGrowthRate("CountryCases.csv","United Kingdom")

linear fit preferred for United Kingdom growth rate 0.0988581653017098


/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


(0.0988581653017098, 0.0, 18.396439308785965)

## List of countries

Now investigate the growth rates in a user-defined list of countries and write to file

In [191]:
#Read in countries from file 
def WriteGrowthRates(country_file):
    
    data = pandas.read_csv(country_file)
    #print(data.columns[0])
        
    csvname = "GrowthRatesAll.csv"

    with open(csvname,mode='w') as growthrate_file:
        csv_writer = csv.DictWriter(growthrate_file,fieldnames=["Country","GrowthRate1","GrowthRate2","DayOfChange"])
        #country_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writeheader()
    
        cid = 1
        while cid < len(data.columns):
            growthrate = ObtainGrowthRate(country_file,data.columns[cid])
            print("Growth rate for",data.columns[cid],"is",growthrate[0])
            csv_writer.writerow({"Country":data.columns[cid],"GrowthRate1":format(growthrate[0],'.5g'),
                                "GrowthRate2":format(growthrate[1],'.5g'),"DayOfChange":format(growthrate[2],'.5g')})
            cid += 1

In [192]:
WriteGrowthRates("CountryCasesAll.csv")

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


Insufficient data for Afghanistan
Growth rate for Afghanistan is 0.0
Insufficient data for Albania
Growth rate for Albania is 0.0
Insufficient data for Algeria
Growth rate for Algeria is 0.0
Insufficient data for Andorra
Growth rate for Andorra is 0.0
Insufficient data for Angola
Growth rate for Angola is 0.0
Insufficient data for Antigua and Barbuda
Growth rate for Antigua and Barbuda is 0.0
linear fit preferred for Argentina growth rate 0.09689376015685949
Growth rate for Argentina is 0.09689376015685949
linear fit preferred for Armenia growth rate 0.050950909512301856
Growth rate for Armenia is 0.050950909512301856
linear fit preferred for Australia growth rate 0.09351583948906489
Growth rate for Australia is 0.09351583948906489
linear fit preferred for Austria growth rate 0.1105524902105001
Growth rate for Austria is 0.1105524902105001
Insufficient data for Azerbaijan
Growth rate for Azerbaijan is 0.0
Insufficient data for Bahamas
Growth rate for Bahamas is 0.0
linear fit preferred

Insufficient data for Moldova
Growth rate for Moldova is 0.0
Insufficient data for Monaco
Growth rate for Monaco is 0.0
Insufficient data for Mongolia
Growth rate for Mongolia is 0.0
Insufficient data for Montenegro
Growth rate for Montenegro is 0.0
Insufficient data for Montserrat
Growth rate for Montserrat is 0.0
Insufficient data for Morocco
Growth rate for Morocco is 0.0
Insufficient data for Mozambique
Growth rate for Mozambique is 0.0
Insufficient data for Myanmar
Growth rate for Myanmar is 0.0
Insufficient data for Namibia
Growth rate for Namibia is 0.0
Insufficient data for Nepal
Growth rate for Nepal is 0.0
linear fit preferred for Netherlands growth rate 0.08972739389366349
Growth rate for Netherlands is 0.08972739389366349
Insufficient data for Netherlands Antilles
Growth rate for Netherlands Antilles is 0.0
Insufficient data for New Caledonia
Growth rate for New Caledonia is 0.0
Insufficient data for New Zealand
Growth rate for New Zealand is 0.0
Insufficient data for Nicar